<a href="https://colab.research.google.com/github/marco-siino/SemEval2024/blob/main/Task%2010/SemEval_2024_Task_10_Mistral7B_MSiino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies. You might need to tweak the CMAKE_ARGS for the `llama-cpp-python` pip package.

In [ ]:
# GPU llama-cpp-python; Starting from version llama-cpp-python==0.1.79, it supports GGUF
!CMAKE_ARGS="-DLLAMA_CUBLAS=on " pip install 'llama-cpp-python>=0.1.79' --force-reinstall --upgrade --no-cache-dir
# For download the models
!pip install huggingface_hub
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 41.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 161.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 210.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.37-cp310-cp310-manylinux_2_35_x86_64.whl size=9593032 sha256=8e59aec7b578008db104a0d2b2c870a0bcc12e44e940854d44325cfbe350c334
  Stored in directory: /tmp/pip-ephem-wheel-cache-1sr62tjw/wheels/5b/4f/36/1df917d3d21e5d04ce960616a6fb9e39c1548149620528a67c
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.

We start by downloading an instruction-finetuned Mistral model, which we will ask to classify model outputs for us.

In [ ]:
from huggingface_hub import hf_hub_download

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

#model_name_or_path='TheBloke/phi-2-GGUF'
#model_basename = "phi-2.Q8_0.gguf"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# This config has been tested on an RTX 3080 (VRAM of 16GB).
# you might need to tweak with respect to your hardware.
from llama_cpp import Llama
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=4, #16, # CPU cores
    n_batch=800, #8000, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=8192, # Context window
    logits_all=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.chat_template': "{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token}}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}", 'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.ggml.padding_token_id': '0', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.rope.freq_base': '1000000.000000', 'llama.context_length': '32768', 'general.na

# Pre-Prompt definitions.

In [ ]:
few_shot_definitions = """

Learn from these Examples the Definitions of the Persuasion Techniques:

1. Presenting Irrelevant Data (Red Herring)
Introducing irrelevant material to the issue being discussed, so that everyone's attention is diverted away from the points made.
Example 1: In politics, defending one’s own policies regarding public safety - “I have worked hard to help eliminate criminal activity. What we need is economic growth that can only come from the hands of leadership.”
Example 2: "You may claim that the death penalty is an ineffective deterrent against crime -- but what about the victims of crime? How do you think surviving family members feel when they see the man who murdered their son kept in prison at their expense? Is it right that they should pay for their son's murderer to be fed and housed?"

2. Misrepresentation of Someone's Position (Straw Man)
When an opponent's proposition is substituted with a similar one which is then refuted in place of the original proposition.
Example: Zebedee: What is your view on the Christian God?
Mike: I don’t believe in any gods, including the Christian one.
Zebedee: So you think that we are here by accident, and all this design in nature is pure chance, and the universe just created itself?
Mike: You got all that from me stating that I just don’t believe in any gods?
Explanation: Mike made one claim: that he does not believe in any gods. From that, we can deduce a few things, like he is not a theist, he is not a practicing Christian, Catholic, Jew, or a member of any other religion that requires the belief in a god, but we cannot deduce that he believes we are all here by accident, nature is chance, and the universe created itself.

3. Whataboutism
A technique that attempts to discredit an opponent's position by charging them with hypocrisy without directly disproving their argument.
Example 1: a nation deflects criticism of its recent human rights violations by pointing to the history of slavery in the United States.
Example 2: "Qatar spending profusely on Neymar, not fighting terrorism"

4. Causal Oversimplification
Assuming a single cause or reason when there are actually multiple causes for an issue.
It includes transferring blame to one person or group of people without investigating the complexities of the issue
Example 1: “President Trump has been in office for a month and gas prices have been skyrocketing. The rise in gas prices is because of President Trump.”
Example 2: The reason New Orleans was hit so hard with the hurricane was because of all the immoral people who live there.
Explanation: This was an actual argument seen in the months that followed hurricane Katrina. Ignoring the validity of the claims being made, the arguer is blaming a natural disaster on a group of people.
Example 3: if France had not have declared war on Germany then world war two would have never happened.

5. Obfuscation, Intentional vagueness, Confusion
Using words which are deliberately not clear so that the audience may have its own interpretations.
For example when an unclear phrase with multiple definitions is used within the argument and, therefore, it does not support the conclusion.
Example: It is a good idea to listen to victims of theft. Therefore if the victims say to have the thief shot, then you should do that.
Explanation: the definition for "listen to" is equivocated here. In the first case it means listen to their personal account of the experience of being a victim of theft. Empathize with them. In the second case "listen to" means carry out a punishment of their choice.

6. Appeal to authority
Stating that a claim is true simply because a valid authority or expert on the issue said it was true, without any other supporting evidence offered. We consider the special case in which the reference is not an authority or an expert in this technique, altough it is referred to as Testimonial in literature.
Example: Richard Dawkins, an evolutionary biologist and perhaps the foremost expert in the field, says that evolution is true. Therefore, it's true.
Explanation: Richard Dawkins certainly knows about evolution, and he can confidently tell us that it is true, but that doesn't make it true. What makes it true is the preponderance of evidence for the theory.
Example 2: "According to Serena Williams, our foreign policy is the best on Earth. So we are in the right direction."
Details: since there is a chance that any authority can be wrong, it is reasonable to defer to an authority to support a claim, but the authority should not be the only justification to accept the claim, otherwise the Appeal-to-Authority fallacy is committed.

7. Black-and-white Fallacy, Dictatorship
Presenting two alternative options as the only possibilities, when in fact more possibilities exist. As an the extreme case, tell the audience exactly what actions to take, eliminating any other possible choices (Dictatorship).
Example 1: You must be a Republican or Democrat. You are not a Democrat. Therefore, you must be a Republican
Example 2: I thought you were a good person, but you weren’t at church today.
Explanation: The assumption here is that if one doesn't attend chuch, one must be bad. Of course, good people exist who don’t go to church, and good church-going people could have had a really good reason not to be in church.
Example 3: There is no alternative to war

8. Name calling or labeling
Labeling the object of the propaganda campaign as either something the target audience fears, hates, finds undesirable or loves, praises.
Examples: "Republican congressweasels", "Bush the Lesser" (note that lesser does not refer to "the second", but it is pejorative)

9. Loaded Language
Using specific words and phrases with strong emotional implications (either positive or negative) to influence an audience.
Example 1: "[...] a lone lawmaker’s childish shouting. ".
Example 2: "how stupid and petty things have become in Washington"

10. Exaggeration or Minimisation
Either representing something in an excessive manner: making things larger, better, worse (e.g., "the best of the best", "quality guaranteed") or making something seem less important or smaller than it really is (e.g., saying that an insult was just a joke).
Example 1: "Democrats bolted as soon as Trump’s speech ended in an apparent effort to signal they can’t even stomach being in the same room as the president "
Example 2: "We’re going to have unbelievable intelligence"
Example 3: I was not fighting with her; we were just playing.

11. Flag-waving
Playing on strong national feeling (or to any group; e.g., race, gender, political preference) to justify or promote an action or idea
Example 1: "patriotism mean no questions" (this is also a slogan)
Example 2: "entering this war will make us have a better future in our country."

12. Doubt
Questioning the credibility of someone or something.
Example: A candidate talks about his opponent and says: Is he ready to be the Mayor?

13. Appeal to fear/prejudice
Seeking to build support for an idea by instilling anxiety and/or panic in the population towards an alternative.
In some cases the support is built based on preconceived judgements.
Example 1: "either we go to war or we will perish" (this is also aBlack and White fallacy))
Example 2: "we must stop those refugees as they are terrorists"

14. Slogans
A brief and striking phrase that may include labeling and stereotyping. Slogans tend to act as emotional appeals.
Example 1: "The more women at war . . . the sooner we win."
Example 2: "Make America great again!"

15. Thought-terminating cliché
Words or phrases that discourage critical thought and meaningful discussion about a given topic. They are typically short, generic sentences that offer seemingly simple answers to complex questions or that distract attention away from other lines of thought.
Examples: It is what it is; It's just common sense; You gotta do what you gotta do; Nothing is permanent except change; Better late than never; Mind your own business; Nobody's perfect; It doesn't matter; You can't change human nature.

16. Bandwagon
Attempting to persuade the target audience to join in and take the course of action because "everyone else is taking the same action".
Example 1: Would you vote for Clinton as president? 57% say yes
Example 2: 90% of citizens support our initiative. You should.

17. Reductio ad hitlerum
Persuading an audience to disapprove an action or idea by suggesting that the idea is popular with groups hated in contempt by the target audience. It can refer to any person or concept with a negative connotation.
Example 1: "Do you know who else was doing that ? Hitler!"
Example 2: "Only one kind of person can think in that way: a communist."

18. Repetition
Repeating the same message over and over again so that the audience will eventually accept it.

19. Smears
A smear is an effort to damage or call into question someone's reputation, by propounding negative propaganda. It can be applied to individuals or groups.

20. Glittering Generalities
These are words or symbols in the value system of the target audience that produce a positive image when attached to a person or issue. Peace, hope, happiness, security, wise leadership, etc.


"""

In [ ]:
categories = """

Persuasion Techniques:

1. Presenting Irrelevant Data (Red Herring)

2. Misrepresentation of Someone's Position (Straw Man)

3. Whataboutism

4. Causal Oversimplification

5. Obfuscation, Intentional vagueness, Confusion

6. Appeal to authority

7. Black-and-white Fallacy, Dictatorship

8. Name calling or labeling

9. Loaded Language

10. Exaggeration or Minimisation

11. Flag-waving

12. Doubt

13. Appeal to fear/prejudice

14. Slogans

15. Thought-terminating cliché

16. Bandwagon

17. Reductio ad hitlerum

18. Repetition

19. Smears

20. Glittering Generalities


"""

In [ ]:
labels_dict = {
               1: 'Presenting Irrelevant Data (Red Herring)',

2: 'Misrepresentation of Someone\'s Position (Straw Man)',

3: 'Whataboutism',

4: 'Causal Oversimplification',

5: 'Obfuscation, Intentional vagueness, Confusion',

6: 'Appeal to authority',

7: 'Black-and-white Fallacy, Dictatorship',

8: 'Name calling or labeling',

9: 'Loaded Language',

10: 'Exaggeration or Minimisation',

11: 'Flag-waving',

12: 'Doubt',

13: 'Appeal to fear/prejudice',

14: 'Slogans',

15: 'Thought-terminating cliché',

16: 'Bandwagon',

17: 'Reductio ad hitlerum',

18: 'Repetition',

19: 'Smears',

20: 'Glittering Generalities'

}

# Run!

In [ ]:
# change paths appropriately
# make sure the output filename is the same as the reference filename for the scoring program
test_data = "en_subtask1_test_unlabeled.json"
result_data = "result.json"

from datasets import load_dataset
import json
import re
import random
import numpy as np
import tqdm.notebook as tqdm
seed_val = 442
random.seed(seed_val)
np.random.seed(seed_val)

# simple JSON loading
with open(test_data, 'r') as istr:
    data_val_all = json.load(istr)
num_sample = len(data_val_all)
print(num_sample)

output_json = []

index = 1051

for i in tqdm.trange(num_sample):
  index+=1
  print("\n",index)
  hyp = str(data_val_all[i]['text'])
  t=hyp.split("\\n")
  hyp=' - '.join(t)
  id = data_val_all[i]['id']
  context = f"{few_shot_definitions}"

  sentence = f"Sentence: {hyp}"
  message = f"\n{context}\n{sentence}\n{categories}\nGiven the above Definitions of the Persuasion Techniques, Identify the Persuasion Techniques used in the Sentence. Answer using ONLY one or more numbers in the range 1-20 separated by commas. No text nor other options are allowed."
  prompt = f"<s>[INST] {message} [/INST]"
  print("\n",sentence)

  response = lcpp_llm(
        prompt=prompt,
        temperature= 0.0,
        logprobs=1,
    )

  answer = str(response["choices"][0]["text"])#.strip().lower()

  print(answer)

  numerical_categories = answer.split(",")
  #print(numerical_categories)
  list_labels = []

  for x in numerical_categories:
    value = re.sub("[^0-9]", "", x)

    if value!='':
      value = int(value)
      if (value>20):
        value = value/10
        value = int(value)
        if (value>20):
          value = value/10
          value = int(value)
      list_labels.append(labels_dict[value])
  item_to_json=[]
  item_to_json = {'id':id,'labels':list_labels}
  output_json.append(item_to_json)

  f = open(result_data, 'w', encoding='utf-8')
  json.dump(output_json, f)
  f.close()
print("done")

447


  0%|          | 0/447 [00:00<?, ?it/s]


 1052

 Sentence: POLITICIANS ARE A DIME A DOZEN -  - TRUE LEADERS ARE PRICELESS!
 8, 9, 14, 20

 1053

 Sentence: DID YOU KNOW? -  - 1 OUT OF 3 TRUMP SUPPORTERS ARE JUST AS STUPID AS THE OTHER 2


Llama.generate: prefix-match hit


 3, 8, 9, 13, 15

 1054

 Sentence: American Children 1950 - American Children 2019


...

 2, 3, 4, 8, 9, 

 1497

 Sentence: IF YOU THINK THESE PUPPETS ARE THE PROBLEM -  - THEN YOU DON'T UNDERSTAND THE PROBLEM


Llama.generate: prefix-match hit


 3, 4, 5 (Causal Oversimplification

 1498

 Sentence: THERE ARE TWO TYPES OF REPUBLICANS: - MILLIONAIRES & SUCKERS


Llama.generate: prefix-match hit


 8, 9, 13, 17 (Name calling
done


In [ ]:
!zip -r results.zip result.json

  adding: result.json (deflated 94%)


In [ ]:
from google.colab import files
files.download('results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>